In [2]:
import torch
import numpy as np
import sys
sys.path.insert(0, '../lib')
from models import TaxonomyCNN, evaluate_model
from dataset_utils import train_test_split, split_dataset
from datasets import Genes, Mutations

In [3]:
args_dict = [{'batch_size': 5, 'epochs': 10, 'lr': 0.01}]

for i in range(len(args_dict)):
    dataset = Genes('../data', k=500)
    dataset, test_set = split_dataset(dataset, 0.1, True)

    train_labels = torch.from_numpy(np.array(dataset.y)).cuda()
    test_labels = torch.from_numpy(np.array(test_set.y))

    dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=5)
    labels_loader = torch.utils.data.DataLoader(train_labels, batch_size=5)

    model = TaxonomyCNN().train().cuda()
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=args_dict[i]['lr'])

    losses = []
    loss = None
    for epoch in range(args_dict[i]['epochs']):
        print('Epoch: ' + str(epoch+1) + '/' + str(args_dict[i]['epochs']))
        model = model.train().cuda()

        for x, y in zip(dataset_loader, labels_loader):
            optimizer.zero_grad()

            prediction = model(x.cuda())
            loss = criterion(prediction, y)

            loss.backward()
            optimizer.step()

        losses.append(loss.item())
        print('Loss: ' + str(losses[-1]))
        test_acc = evaluate_model(model.cpu(), test_set, test_labels)
        print('Test accuracy: ' + str(test_acc) + '\n')

    torch.save({'state_dict': model.state_dict, 'loss': losses, 'args_dict': args_dict[i]}, 'taxonomy_cnn.pt')



Epoch: 1/10


/home/rozver/Documents/RSI/RSI-2021-HT/research/../lib/models.py:65: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(x)


Loss: 1.7839276790618896
Test accuracy: 0.15566666666666668

Epoch: 2/10
Loss: 1.7864129543304443
Test accuracy: 0.17033333333333334

Epoch: 3/10
Loss: 1.7921209335327148
Test accuracy: 0.17266666666666666

Epoch: 4/10
Loss: 1.792467474937439
Test accuracy: 0.19433333333333333

Epoch: 5/10
Loss: 1.792729139328003
Test accuracy: 0.234

Epoch: 6/10
Loss: 1.8315885066986084
Test accuracy: 0.261

Epoch: 7/10
Loss: 1.7907569408416748
Test accuracy: 0.265

Epoch: 8/10
Loss: 1.7247190475463867
Test accuracy: 0.2653333333333333

Epoch: 9/10
Loss: 1.8087753057479858
Test accuracy: 0.2623333333333333

Epoch: 10/10
Loss: 1.854079008102417
Test accuracy: 0.249



In [11]:
    train_acc = evaluate_model(model.cpu(), dataset, train_labels.cpu())
    print('Final train accuracy: ' + str(train_acc))
    
    test_labels = torch.from_numpy(np.asarray(test_labels))
    test_acc = evaluate_model(model.cpu(), test_set, test_labels)
    print('Final test accuracy: ' + str(test_acc))

Final train accuracy: 0.2598888888888889
Final test accuracy: 0.26266666666666666
